In [9]:
import os
import re
import json
import pandas as pd
from src.utils import read_config
from prompts import gemini_prompts
import google.generativeai as genai
import google.ai.generativelanguage as glm
from typing import Any, Dict

from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS, DistanceStrategy
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## 0609 notes
1. Add KG(knowledge graph) as part of metadata
2. Use SelfQueryRetriver to get the top_k answer
3. Override the `_get_docs_with_query` to use the `similarity_search_with_score` method(or with `similarity_search_with_score_by_vector`)

In [10]:
configs = read_config('.env/configs.json')
api_key =configs['g_key']
os.environ["GOOGLE_API_KEY"] = api_key
genai.configure(api_key=api_key)

In [11]:
model_name = "sentence-transformers/distiluse-base-multilingual-cased-v1"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
# vector_db_path = "./vector_db_2/"

# # load FAISS
# vectorstore = FAISS.load_local(folder_path=vector_db_path, allow_dangerous_deserialization=True, embeddings=embeddings)

In [13]:
# chromadb
vector_db_path = "./chroma_db/"
chroma_collection_name = "correction_cosine_v1"
vectorstore = Chroma(persist_directory=vector_db_path, collection_name=chroma_collection_name, embedding_function=embeddings)

In [14]:
vectorstore._collection.count()

242

### Function for getting triple

In [15]:
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    convert_system_message_to_human=True,
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
)

In [16]:
# top_k = 3
# retriever = SelfQueryRetriever.from_llm(
#     llm,
#     vectorstore,
#     document_contents="test",
#     metadata_field_info=metadata_field_info,
#     k=top_k,
#     search_type='similarity',
#     search_kwagrs={"k":top_k},
#     enable_limit=True
# )

# additional query
# mixed_search_string = additional_query + "與" + query

In [17]:
CONCLUSION_PROMPT="""
### INSTRUCTION: 說明查詢的內容 QUERY 與 REF 中的之間的關係。若找不到合理的關係則說沒有明確的相關性，不要亂說
### QUERY: {query}
### REF: {reference}
### ASSISTANT: 
"""

In [18]:
# response_data = {}
# for i, page in enumerate(result):
#     print(page.metadata['source'])
#     # summary_prompt = PromptTemplate.from_template(gemini_prompts.SUMMARY_PROMPT)
#     # chain = summary_prompt | llm
#     # summary_data = {
#     #     "reference": page.page_content
#     # }

#     # summary_response = chain.invoke(summary_data)
    
#     # pattern = r'\*\*摘要：\*\*'
#     # if re.search(pattern, summary_response.content):
#     #     re.sub(pattern, '', summary_response.content)
        
#     suffix = "### ASSISTANT: "
#     input_variables = ["query", "reference"]

#     conclusion_prompt = PromptTemplate.from_examples(CONCLUSION_PROMPT, suffix, input_variables)
#     chain = conclusion_prompt | llm
#     conclusion_data = {
#         "query": query,
#         "reference": page.page_content
#     }
#     conclusion_response = chain.invoke(conclusion_data)

#     # response_data[i] = {
#     #     "summary": summary_response.content,
#     #     "source": page.metadata['source'],
#     #     "score": f"{page.metadata['score']:.2f}",
#     #     "target": page.metadata['target'],
#     #     # "conclusion": conclusion_response.content
#     # }
    



In [19]:
metadata_field_info = [
    AttributeInfo(
        name='target',
        description='對象、受文對象、受文者',
        type='string',
    ),
    AttributeInfo(
        name='reason',
        description='被糾正原因、案由',
        type='string',
    ),
    AttributeInfo(
        name='fact',
        description='被糾正的證據、事實、事實與理由',
        type='string',
    ),
    AttributeInfo(
        name='keywords',
        description='被糾正原因中的關鍵字',
        type='string'
    ),
    AttributeInfo(
        name='relationship_between_entities',
        description='被糾正原因中呈現三元組關係的entities(實體)',
        type='string'
    )
]

In [20]:
query = '台灣糖業公司'

In [21]:
class CustomSelfQueryRetriever(SelfQueryRetriever):
        def _get_docs_with_query(
            self, query: str, search_kwargs: Dict[str, Any]
        ):
            """Get docs, adding score information."""
            docs, scores = zip(
                *vectorstore.similarity_search_with_relevance_scores(query, **search_kwargs)
            )
            for doc, score in zip(docs, scores):
                doc.metadata["score"] = score
                # if score < 1:
                #     doc.metadata["score"] = 1-score
                # elif score > 1:
                #     doc.metadata["score"] = 0.001

            return docs
        
document_content_description = 'agent'
retriever = CustomSelfQueryRetriever.from_llm(
        llm,
        vectorstore,
        document_content_description,
        metadata_field_info,
        verbose=True
)

docs = retriever.get_relevant_documents(query)


c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


In [22]:
for d in docs:
    print(d.metadata['target'])

台灣糖業股份有限公司
衛生福利部食品藥物管理署
台灣電力股份有限公司
勞動部


In [23]:
for d in docs:
    print(d.metadata['target'])

台灣糖業股份有限公司
衛生福利部食品藥物管理署
台灣電力股份有限公司
勞動部


In [26]:
class CustomSelfQueryRetriever(SelfQueryRetriever):
        def _get_docs_with_query(
            self, query: str, search_kwargs: Dict[str, Any]
        ):
            """Get docs, adding score information."""
            docs, scores = zip(
                *vectorstore.similarity_search_with_relevance_scores(query, **search_kwargs)
            )
            for doc, score in zip(docs, scores):
                doc.metadata["score"] = score
                # if score < 1:
                #     doc.metadata["score"] = 1-score
                # elif score > 1:
                #     doc.metadata["score"] = 0.001

            return docs
        
document_content_description = 'agent'
retriever = CustomSelfQueryRetriever.from_llm(
        llm,
        vectorstore,
        document_content_description,
        metadata_field_info,
        verbose=True
)
query2 = "對象包括" + query
print(query2)
docs = retriever.get_relevant_documents(query2)

response_data = {}
for i, page in enumerate(docs):
    print(page.metadata['source'])
    summary_prompt = PromptTemplate.from_template(gemini_prompts.SUMMARY_PROMPT)
    chain = summary_prompt | llm
    summary_data = {
        "reference": page.page_content
    }

    summary_response = chain.invoke(summary_data)
    
    pattern = r'\*\*摘要：\*\*'
    if re.search(pattern, summary_response.content):
        re.sub(pattern, '', summary_response.content)
        
        
    # conclusion_prompt = PromptTemplate.from_examples(gemini_prompts.CONCLUSION_PROMPT)
    # chain = conclusion_prompt | llm
    # conclusion_data = {
    #     "query" : query,
    #     "reference": page.page_content
    # }
    # conclusion_response = chain.invoke(conclusion_data)



    response_data[i] = {
        "summary": summary_response.content,
        "source": page.metadata['source'],
        "score": f"{page.metadata['score']:.2f}",
        "target": page.metadata['target'],
        # "conclusion": conclusion_response.content
    }

對象包括台灣糖業公司


c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


1120607-台糖七股農場造林地改置光電場案糾正案文(決議版)_20231211131825677.pdf


c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


糾正案文_ (62).pdf


c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


台電虛報加班費案-糾正案文-公布版_20230615082314856.pdf


c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


糾正案文_ (49).pdf


c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


In [31]:
page.page_content

'由勞動部勞動基金運用局（下稱勞金局）於民國（下同）109年間爆發該局國內投資組前組長游廼文與寶○集團人員、受委託經營投信業者共同炒作遠○○○股份有限公司股票之違失，該局既知游員多次遭檢舉，並於105年將之列入廉政風險人員名單之中，詎仍讓其擔任國內投資組組長重要職務長達5年餘之久，未將其調整或調離職務，以發揮防杜功能，亦無更嚴密之監控機制，致游員主導該特定業務領域多年，得以培植並濫用其對投信業者之影響力，進而合謀不正利益，聯合炒作遠○公司股票，造成基金損失高達新臺幣（下同）2,769萬餘元，雖已獲償付，然已引發勞動基金信心危機，斲傷政府形象；又該局未嚴予督促所屬確實依規落實執行交易室管理注意事項，致該注意事項形同具文，且漠視交易室安全管理，交易室淪作儲藏室、休息室使用；復就勞動基金監理委員會委員提出該注意事項未能落實及未臻健全等問題，亦延宕處理，未能積極因應改善，遲至本案爆發，外界訾議批評聲浪不斷後，方為強化內控機制，檢討提出強化措施，修訂注意事項，以作更嚴格之控管，核其所為，實均難卸怠失之責，爰依法提案糾正2'

In [27]:
response_data

{0: {'summary': '台灣糖業公司配合政府推動太陽能光電計畫，將造林地改為光電用地，但此舉與政府原先推動造林政策相矛盾，引發民眾疑慮和爭議。\n\n此外，該公司在估算林木材積和伐採作業方面，程序不夠嚴謹，缺乏佐證資料，導致無法驗證履約管理。審計部認定公司有重大違失，已提案糾正。',
  'source': '1120607-台糖七股農場造林地改置光電場案糾正案文(決議版)_20231211131825677.pdf',
  'score': '0.27',
  'target': '台灣糖業股份有限公司'},
 1: {'summary': '衛生福利部食品藥物管理署制定了蜂蜜標示規定，要求蜂蜜含量達 60% 的產品標示為「蜂蜜」。由於缺乏蜂蜜含量檢驗方法，該署從未抽驗市售蜂蜜產品，無法確保產品標示準確。因此，該規定形同虛設，顯示該署制定規定時草率疏失，特此提案糾正。',
  'source': '糾正案文_ (62).pdf',
  'score': '0.22',
  'target': '衛生福利部食品藥物管理署'},
 2: {'summary': '台灣電力公司電力修護處辦理電廠大修或檢修工作時，委託協力廠商人員加班和門禁資料異常達13,863筆。儘管疑似虛報金額已從3,298萬元降至255萬元，但台電未依合約懲處虛報價金廠商。\n\n此外，電力修護處員工支援電廠大修時，因公務繁忙未刷卡，卻未補登，且將此歸咎於人工作業疏失，導致加班費和補休發放浮濫。\n\n更甚者，電廠身為國家關鍵基礎設施，電力修護處員工卻有代他人刷卡、協力廠商人員未遵守門禁管制等違規行為。',
  'source': '台電虛報加班費案-糾正案文-公布版_20230615082314856.pdf',
  'score': '0.18',
  'target': '台灣電力股份有限公司'},
 3: {'summary': '勞動部勞金局爆發炒股違失，前組長游廼文與寶○集團及投信業者合謀炒作遠○公司股票，造成基金損失超過 2,700 萬元。儘管資金已獲償，但已損及勞動基金信心和政府形象。\n\n勞金局明知游廼文有違規紀錄，卻仍讓他擔任重要職務 5 年，未採取防範措施。交易室管理注意事項未落實，淪為儲藏室和休息室。\n\n勞動基金監理委員會委員提出的問題，勞金局遲未改善，直到案

In [49]:
page.metadata

{'fact': '民國(下同)103年6月27日，國防部陸軍司令部後勤指揮部(下稱陸勤部)湳湖營區，疑未依程序銷毀逾期彈藥，致發生爆炸意外，3名承包商作業人員未著防護裝備，遭嚴重灼傷，其中1人傷重不治，爰經本院國防及情報委員會決議推派調查案經本院向國防部、陸勤部、行政院公共工程委員會（下稱行政院工程會）、勞動部、新竹縣政府暨所屬消防局等機關調閱有關卷證資料、法令詳於審閱；並針對本案調卷及案情相關疑點，請相關單位主管及承辦人員說明，並提供本案廢彈處理合約暨履約相關內容等全卷及職災相關檢查報告供參；復於104年2月4日詢問國防部後勤參謀次長室次長尚○○中將、助理次長崔○○少將、陸軍司令部參謀長郝○○中將、彈藥處處長簡○○少將、軍備局獲得管理處處長歐陽○○少將及相關承辦人等到院說明，該部嗣又先後於同年2月13日、3月18日及5月11日檢陳詢問補充說明及本案相關起訴與後續查證說明等資料到院，業調查竣事，茲臚列調查2發現之違失如次一、陸勤部本案未依政府採購法履約管理採購契約相關法令規定切實辦理，核有違失；且衡其對本案廢彈委商處理履約督管機制未臻健全及完善，致衍生本案爆炸傷亡重大職業災害悲劇憾事，自屬難辭其咎，亦核有明顯缺失，以上均核有違失，應切實檢討改進(一)經查，有關本案委商處理履約督管乙節，按行政院工程會所訂頒政府採購法第四章「履約管理」第63條規定「各類採購契約以採用主管機關訂定之範本為原則，其要項及內容由主管機關參考國際及國內慣例定之」同法第70條規定「機關辦理工程採購，應明訂廠商執行品質管理、環境保護、施工安全衛生之責任，並對重點項目訂定檢查程序及檢驗標準（第1項）機關於廠商履約過程，得辦理分段查驗，其結果並得供驗收之用（第2項）…財物或勞務採購需經一定履約過程，而非以現成財物或勞務供應者，準用第1項及第2項之規定（第5項）」另依該會所訂頒「公共工程施工品質管理作業要點」第15點規定「機關應隨時督導工程施工情形，並留存紀錄備查另得視工程需要設置工程督導小組，隨時進行施工品質督導工作機關發現工程缺失時，應即以書面通知監造單位或廠商限期改善」以上規定甚明(二)另依本案「勞動部重大職業災害檢查報告書」涉此內容摘錄該報告第十二項、「本災害構成其他法律罰則事項」-(三)關係事業單位（國家中山科學研究院系統製造中心）「該事業單位承攬國防部『國軍逾期傳統批號彈藥委商處理(契約編

In [84]:
query

'三元組中包含郵政法規定之郵件補償範疇'